In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from typing import Any, Type
from langchain_community.tools import WikipediaQueryRun, GoogleSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, GoogleSearchAPIWrapper

TA's 힌트
다양한 커스텀 도구를 만들고 이 도구들을 에이전트에게 전달하여 리서치 AI 에이전트를 구현해야 하는 챌린지입니다.

먼저, 위키피디아에서 검색을 수행하는 도구를 만들어야 합니다. 랭체인에서 제공하는 WikipediaQueryRun를 사용하면 간단하게 구현할 수 있습니다. (LangChain - Wikipedia 를 참고하세요.)

다음은 DuckDuckGo에서 검색을 수행하는 도구를 만들어야 합니다. 이 또한 랭체인에서 제공하는 DuckDuckGoSearchRun를 사용하면 덕덕고의 검색 결과를 얻을 수 있습니다. (LangChain - DuckDuckGo Search 를 참고하세요.)

DuckDuckGo를 통해 얻은 검색 결과에서 웹사이트들의 콘텐츠를 추출하는 도구가 필요합니다. 다양한 방법이 있지만, 그중에서도 랭체인이 제공하는 WebBaseLoader를 이용하면 웹사이트의 콘텐츠를 쉽게 스크랩할 수 있습니다. (LangChain - WebBaseLoader 를 참고하세요.)

마지막으로, 리서치 결과를 txt 파일에 저장하는 도구가 필요합니다. 파이썬에서 기본으로 제공하는 파일 쓰기 기능을 이용하여 구현하세요. (python - reading-and-writing-files)

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

In [3]:
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for Example query: Search for Apple Company."
    )


class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = (
        "Use this tool to find any information for the query. It takes a query as an argument. And save the results in file."
    )
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)

In [4]:
class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search websites about Example query: Search websites about Apple Company."
    )


class SearchTool(BaseTool):
    name: str = "SearchTool"
    description: str = (
        "Use this tool to find websites relate about the query. It takes a query as an argument. And save the results in file."
    )
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        google = GoogleSearchAPIWrapper()
        return google.run(query)

In [5]:
class SaveToolArgsSchema(BaseModel):
    filename: str = Field(description="Filename to save the search results.")
    contents: str = Field(description="the search results.")


class SaveTool(BaseTool):
    name: str = "SaveTool"
    description: str = "Use this tool to save search results in text file."
    args_schema: Type[SaveToolArgsSchema] = SaveToolArgsSchema

    def _run(self, filename, contents):
        # print(filename, contents)
        f = open(filename, "w", encoding="utf-8")
        f.write(contents)
        f.close()

In [ ]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        SearchTool(),
        SaveTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)